<a href="https://colab.research.google.com/github/ssimpore/ParcInfos/blob/master/ManageData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import pandas as pd
import pickle
import os.path

In [118]:
class Parametre:
    def __init__(self):
        self.list_app = ['INFOS', 'ALERTE']
        self.list_option = ['Informations générales', 'Informations ACR (ENEDIS)', 'Informations COM (IP)','Informations autres']
        self.label_col1 = ['Parc', 'Agence', 'Exploitant', 'Responsable', 'SCADA', 'Maintenance','N° mainteneur', 'Longitude']
        self.label_col2 = ['Technologie', 'N° astreinte', 'N° exploitant', 'N° responsable', 'Statut parc','Mainteneur','N° astreinte maint.', 'Latitude']

    def __repr__(self):
        msg = f"Label d'affichage"
        return msg



In [119]:
class Data:
    verify_name = 'None'
    def verify(df, nom_parc, variable):
        if nom_parc in list(df.index):
            r = df.loc[nom_parc, variable]
        else:
            r = Data.verify_name
        if (str(r) != 'nan') and (r != Data.verify_name):
            x = r
        else:
            x = Data.verify_name
        return x

    def __init__(self, path, new=False):
      update = new or (not os.path.isfile('data'))
      if update:
        self.path = path
        self.dfparc = pd.read_excel(self.path, sheet_name='Parc', index_col=1)
        self.dfagence = pd.read_excel(self.path, sheet_name='Agence', index_col=0)
        self.dfresponsable = pd.read_excel(self.path, sheet_name='Responsable', index_col=0)
        self.dfexploitant = pd.read_excel(self.path, sheet_name='Exploitant', index_col=0)
        self.dfmainteneur = pd.read_excel(self.path, sheet_name='Mainteneur', index_col=0)
        self.dfsolar = self.dfparc[self.dfparc['TECHNOLOGIE'] == 'SOLAR']
        self.dfwind = self.dfparc[self.dfparc['TECHNOLOGIE'] == 'WIND']
        self.param=Parametre()
        
        self.saveDataAsPickle()
      else:
        data = self.useDataFromPickle()
        self.dfparc = data.dfparc
        self.dfagence = data.dfagence
        self.dfresponsable = data.dfresponsable
        self.dfexploitant = data.dfexploitant
        self.dfmainteneur = data.dfmainteneur
        self.dfsolar = data.dfsolar
        self.dfwind = data.dfwind
        self.param = data.param

      
        
        
       
    def saveDataAsPickle(self):
        with open('data', 'wb') as f:
            pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)
        #print('using New data')

    def useDataFromPickle(self):
        with open('data', 'rb') as f:
            data = pickle.load(f)
        #print('using Pickle data')
        return data

    def __repr__(self):
        return f"Taille :{len(self.dfparc)} éléments"

        
class Exploitant(Data):
    def __init__(self, nom_parc, new=False):
        Data.__init__(self, new)
        self.nom = Data.verify(self.dfparc, nom_parc, 'EXPLOITANT')
        self.telephone = Data.verify(self.dfexploitant, self.nom, 'TELEPHONE')
        self.email = Data.verify(self.dfexploitant, self.nom, 'EMAIL')

    def __repr__(self):
        msg = f"Exploitant:\nNom: {self.nom}\nTel: {self.telephone}\nEmail: {self.email}"
        return msg

class Responsable(Data):
    def __init__(self, nom_parc, new=False):
        Data.__init__(self, new)
        self.agence = Data.verify(self.dfparc, nom_parc, 'AGENCE')
        self.nom = Data.verify(self.dfresponsable, self.agence, 'NOM')
        self.telephone = Data.verify(self.dfresponsable, self.agence, 'TELEPHONE')
        self.email = Data.verify(self.dfresponsable, self.agence, 'EMAIL')

    def __repr__(self):
        msg = f"Responsable:\nNom: {self.nom}\nTel: {self.telephone}\nEmail: {self.email}"
        return msg

class Agence(Data):
    def __init__(self, nom_parc, new=False):
        Data.__init__(self, new)
        self.nom = Data.verify(self.dfparc, nom_parc, 'AGENCE')
        self.responsable = Data.verify(self.dfresponsable, self.nom, 'NOM')
        self.astreinte = Data.verify(self.dfagence, self.nom, 'N° ASTREINTE')
        self.email = Data.verify(self.dfagence, self.nom, 'EMAIL')

    def __repr__(self):
        msg = f"Agence:\nNom: {self.nom}\nAstreinte: {self.astreinte}\nEmail: {self.email}"
        return msg


class Mainteneur(Data):
    class Niveau(Data):
        def __init__(self, nom, new=False):
            Data.__init__(self, new)
            self.nom = nom
            self.telephone = Data.verify(self.dfmainteneur, self.nom, 'TELEPHONE')
            self.email_entreprise = Data.verify(self.dfmainteneur, self.nom, 'EMAIL ENTREPRISE')
            self.email_referent = Data.verify(self.dfmainteneur, self.nom, 'EMAIL REFERENT')
            self.astreinte = Data.verify(self.dfmainteneur, self.nom, 'N° ASTREINTE')
            self.num_referent = Data.verify(self.dfmainteneur, self.nom, 'N° REFERENT')

        def __repr__(self):
            msg = f"Nom: {self.nom}\nAstreinte: {self.astreinte}\nEmail: {self.email_entreprise}"
            return msg

class Parc(Data):
    def verify(df, nom):
        if nom in list(df.index):
            r = nom
        else:
            r = Data.verify_name
        return r

    def __init__(self, nom_parc, new=False):
        Data.__init__(self, new)
        self.nom = Parc.verify(self.dfparc, nom_parc)
        self.trigramme = Data.verify(self.dfparc, self.nom, 'CODE')
        self.technologie = Data.verify(self.dfparc, self.nom, 'TECHNOLOGIE')
        self.statut = Data.verify(self.dfparc, self.nom, 'STATUT')
        self.agregation = Data.verify(self.dfparc, self.nom, 'AGRÉGATION')
        self.recette = Data.verify(self.dfparc, self.nom, 'RÉCETTE À FAIRE')
        self.cardi = Data.verify(self.dfparc, self.nom, 'N°CARD-I')
        self.pdl = Data.verify(self.dfparc, self.nom, 'POSTE DE LIVRAISON')
        self.psource = Data.verify(self.dfparc, self.nom, 'POSTE SOURCE')
        self.depart = Data.verify(self.dfparc, self.nom, 'DÉPART')
        self.num_acr = Data.verify(self.dfparc, self.nom, 'N° ACR')
        self.server = Data.verify(self.dfparc, self.nom, 'SERVEUR PARC')
        self.ip_sdrt = Data.verify(self.dfparc, self.nom, 'IP SDRT/RIO PDL')
        self.ip_asa = Data.verify(self.dfparc, self.nom, 'IP ASA')
        self.ip_adsl = Data.verify(self.dfparc, self.nom, 'IP ADSL')
        self.num_adsl = Data.verify(self.dfparc, self.nom, 'N° ADSL')
        self.ip_satellite = Data.verify(self.dfparc, self.nom, 'IP SATELLITE')
        self.commentaire = Data.verify(self.dfparc, self.nom, 'COMMENTAIRE')
        self.scada = Data.verify(self.dfparc, self.nom, 'SCADA')
        self.latitude = Data.verify(self.dfparc, self.nom, 'LATITUDE')
        self.longitude = Data.verify(self.dfparc, self.nom, 'LONGITUDE')

        self.exploitant = Exploitant(self.nom)
        self.responsable = Responsable(self.nom)
        self.agence = Agence(self.nom)
        self.mainteneur = Mainteneur(self.nom)

    def __repr__(self):
        msg = f"Nom :{self.nom} ({self.trigramme})"
        return msg


In [121]:
x=Data('data.xlsx',1)
x.dfparc.head()
e=Parc('Arsac 1')
x.param.list_app

['INFOS', 'ALERTE']